# Day 1: CUDA Dynamic Parallelism Fundamentals

## 🎯 The Manager Who Spawns Sub-Teams On-the-Fly

> **Hook:** Imagine a construction site manager who, upon seeing the actual terrain, can instantly hire specialized sub-teams without calling headquarters. Need extra plumbers for unexpected pipes? Spawn them on-site. Found bedrock? Summon the drilling crew immediately. Traditional CUDA is like calling the office (CPU) every time you need more workers. **Dynamic Parallelism lets the GPU manager spawn new teams directly from the job site!**

---

## 🎯 Learning Objectives

By the end of this session, you will:
1. **Understand CDP architecture** - How parent kernels launch child kernels directly on GPU
2. **Master compilation requirements** - Use `-rdc=true` and `-lcudadevrt` flags correctly
3. **Apply memory visibility rules** - Know what's shared between parent and children
4. **Implement device-side synchronization** - Use `cudaDeviceSynchronize()` effectively
5. **Create device streams** - Enable concurrent child kernel execution

---

## 📚 Prerequisites
- Week 14: PyTorch CUDA Extensions (understanding GPU kernel workflows)
- Understanding of CUDA streams and synchronization
- Familiarity with basic kernel launching from host

In [ ]:
%%writefile cdp_basics.cu
#include <stdio.h>
#include <cuda_runtime.h>

// Child kernel - launched from GPU
__global__ void childKernel(int* data, int offset, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        data[offset + idx] *= 2;  // Double the value
    }
}

// Parent kernel - launches child kernels
__global__ void parentKernel(int* data, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    
    // Only thread 0 launches children
    if (tid == 0) {
        printf("Parent: Launching child kernels from GPU!\n");
        
        int chunkSize = n / 4;
        int threadsPerBlock = 64;
        int blocks = (chunkSize + threadsPerBlock - 1) / threadsPerBlock;
        
        // Launch 4 child kernels, each processing a chunk
        for (int i = 0; i < 4; i++) {
            childKernel<<<blocks, threadsPerBlock>>>(data, i * chunkSize, chunkSize);
        }
        
        // Wait for all children to complete
        cudaDeviceSynchronize();
        printf("Parent: All children completed!\n");
    }
}

int main() {
    const int N = 256;
    int *h_data, *d_data;
    
    h_data = (int*)malloc(N * sizeof(int));
    cudaMalloc(&d_data, N * sizeof(int));
    
    // Initialize data
    for (int i = 0; i < N; i++) h_data[i] = i;
    cudaMemcpy(d_data, h_data, N * sizeof(int), cudaMemcpyHostToDevice);
    
    printf("Before: data[0]=%d, data[100]=%d\n", h_data[0], h_data[100]);
    
    // Launch parent kernel (which launches children)
    parentKernel<<<1, 1>>>(d_data, N);
    cudaDeviceSynchronize();
    
    cudaMemcpy(h_data, d_data, N * sizeof(int), cudaMemcpyDeviceToHost);
    printf("After:  data[0]=%d, data[100]=%d\n", h_data[0], h_data[100]);
    
    cudaFree(d_data);
    free(h_data);
    return 0;
}

## 💡 Concept Card: CDP as Manager Delegation

```
┌─────────────────────────────────────────────────────────────────────────────┐
│  🏗️ THE ON-SITE MANAGER ANALOGY                                            │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  TRADITIONAL CUDA (Calling Headquarters)        CDP (On-Site Authority)    │
│  ───────────────────────────────────────        ────────────────────────    │
│                                                                             │
│  GPU Thread: "Found 4 problem areas!"          GPU Thread: "Found 4 areas!"│
│       │                                              │                      │
│       ▼                                              ▼                      │
│  Return to CPU ────► "Send 4 teams"            Spawn 4 child kernels ◄─────│
│       │                                              │         directly     │
│       ▼                                              ▼                      │
│  CPU launches 4 kernels                        Children work immediately   │
│       │                                              │                      │
│       ▼ (latency!)                                   ▼ (no latency!)       │
│  Wait for completion                           Parent waits on GPU         │
│                                                                             │
├─────────────────────────────────────────────────────────────────────────────┤
│  🔑 KEY INSIGHT:                                                            │
│  • Parent kernel = Site Manager with hiring authority                       │
│  • Child kernels = Sub-teams spawned for specific tasks                     │
│  • cudaDeviceSynchronize() = Manager waiting for sub-teams to finish        │
│  • Device streams = Multiple sub-teams working in parallel                  │
│                                                                             │
│  📋 When to Use CDP:                                                        │
│  • Work discovered AT RUNTIME (irregular workloads)                         │
│  • Recursive algorithms (divide and conquer)                                │
│  • Adaptive mesh refinement                                                 │
│  • Tree/graph traversals                                                    │
└─────────────────────────────────────────────────────────────────────────────┘
```

---

## 🔧 Section 1: Basic CDP - Parent Launching Children

In [ ]:
# Compile with relocatable device code and device runtime
!nvcc -rdc=true -lcudadevrt cdp_basics.cu -o cdp_basics && ./cdp_basics

## Key Compilation Flags

- `-rdc=true`: Enable relocatable device code (required for CDP)
- `-lcudadevrt`: Link device runtime library

## Memory Visibility Rules

1. **Global Memory**: Visible to all grids (parent and children)
2. **Shared Memory**: NOT visible across grids
3. **Local Memory**: Private to each thread
4. **Unified Memory**: Preferred for CDP (automatic coherence)

In [ ]:
%%writefile cdp_memory.cu
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void childModify(int* globalData, int parentValue) {
    int tid = threadIdx.x;
    
    // Child can READ parent's local variable (passed by value)
    printf("Child[%d]: Received parentValue = %d\n", tid, parentValue);
    
    // Child can MODIFY global memory
    globalData[tid] = parentValue + tid;
}

__global__ void parentKernel(int* globalData) {
    __shared__ int sharedData[32];
    int localVar = 42;
    
    sharedData[threadIdx.x] = threadIdx.x * 10;
    __syncthreads();
    
    if (threadIdx.x == 0) {
        // Pass value to child (NOT pointer to shared/local!)
        childModify<<<1, 4>>>(globalData, localVar);
        cudaDeviceSynchronize();
        
        // Check child's modifications
        printf("Parent: globalData = [%d, %d, %d, %d]\n",
               globalData[0], globalData[1], globalData[2], globalData[3]);
    }
}

int main() {
    int *d_data;
    cudaMalloc(&d_data, 32 * sizeof(int));
    cudaMemset(d_data, 0, 32 * sizeof(int));
    
    parentKernel<<<1, 32>>>(d_data);
    cudaDeviceSynchronize();
    
    cudaFree(d_data);
    return 0;
}

In [ ]:
!nvcc -rdc=true -lcudadevrt cdp_memory.cu -o cdp_memory && ./cdp_memory

## Device-Side Synchronization

| Function | Scope | Use Case |
|----------|-------|----------|
| `cudaDeviceSynchronize()` | Wait for ALL children | Most common |
| `cudaStreamSynchronize(stream)` | Wait for stream | Ordered execution |
| Implicit sync at parent exit | Automatic | Default behavior |

In [ ]:
%%writefile cdp_streams.cu
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void worker(int id, int* result) {
    printf("Worker %d running on GPU\n", id);
    result[id] = id * 100;
}

__global__ void coordinator(int* results) {
    cudaStream_t streams[4];
    
    // Create device-side streams
    for (int i = 0; i < 4; i++) {
        cudaStreamCreateWithFlags(&streams[i], cudaStreamNonBlocking);
    }
    
    // Launch workers into different streams (concurrent execution)
    for (int i = 0; i < 4; i++) {
        worker<<<1, 1, 0, streams[i]>>>(i, results);
    }
    
    // Wait for all streams
    cudaDeviceSynchronize();
    
    // Cleanup streams
    for (int i = 0; i < 4; i++) {
        cudaStreamDestroy(streams[i]);
    }
    
    printf("Results: [%d, %d, %d, %d]\n",
           results[0], results[1], results[2], results[3]);
}

int main() {
    int *d_results;
    cudaMalloc(&d_results, 4 * sizeof(int));
    
    coordinator<<<1, 1>>>(d_results);
    cudaDeviceSynchronize();
    
    cudaFree(d_results);
    return 0;
}

In [ ]:
!nvcc -rdc=true -lcudadevrt cdp_streams.cu -o cdp_streams && ./cdp_streams

---

## 🎯 Exercises

### 🔷 CUDA C++ Exercises (Primary)

Implement a parallel sum using dynamic parallelism:
1. Parent divides array into chunks
2. Each child computes partial sum
3. Parent combines partial sums

In [ ]:
%%writefile cdp_fundamentals_exercises.cu
#include <stdio.h>
#include <cuda_runtime.h>

/*
 * CDP Fundamentals Exercises
 * Compile: nvcc -arch=sm_75 -rdc=true -lcudadevrt -o cdp_fundamentals_exercises cdp_fundamentals_exercises.cu
 * 
 * Exercise 1: Parallel Sum with CDP
 * - Parent kernel divides array into chunks
 * - Child kernels compute partial sums
 * - Parent combines results
 * 
 * Exercise 2: Hierarchical Reduction
 * - Implement multi-level reduction using CDP
 * - Each level reduces by factor of 4
 * 
 * Exercise 3: Dynamic Work Distribution
 * - Parent kernel discovers work items
 * - Launch child kernels based on runtime conditions
 */

#define CHECK_CUDA(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            printf("CUDA error at %s:%d: %s\n", __FILE__, __LINE__, \
                   cudaGetErrorString(err)); \
            exit(EXIT_FAILURE); \
        } \
    } while(0)

// =============================================================================
// Exercise 1: Parallel Sum with CDP
// =============================================================================

// Child kernel: sum a chunk of the array
__global__ void sumChunkKernel(float* data, int start, int count, float* partialSum) {
    // TODO: Implement chunk summation
    // Use shared memory for reduction within block
    // Write result to partialSum[blockIdx.x]
    
    extern __shared__ float sdata[];
    int tid = threadIdx.x;
    int idx = start + blockIdx.x * blockDim.x + threadIdx.x;
    
    // Load into shared memory
    sdata[tid] = (idx < start + count) ? data[idx] : 0.0f;
    __syncthreads();
    
    // Reduction in shared memory
    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) {
            sdata[tid] += sdata[tid + s];
        }
        __syncthreads();
    }
    
    if (tid == 0) {
        partialSum[blockIdx.x] = sdata[0];
    }
}

// Parent kernel: coordinate parallel sum
__global__ void parallelSumKernel(float* data, int n, int numChunks, float* result) {
    // TODO: Implement parent kernel
    // 1. Allocate device memory for partial sums
    // 2. Launch child kernels for each chunk
    // 3. Synchronize and combine results
    
    if (threadIdx.x == 0 && blockIdx.x == 0) {
        int chunkSize = (n + numChunks - 1) / numChunks;
        
        // Allocate partial sums (device-side allocation)
        float* partialSums;
        cudaMalloc(&partialSums, numChunks * sizeof(float));
        
        int threadsPerBlock = 256;
        
        // Launch child kernels for each chunk
        for (int i = 0; i < numChunks; i++) {
            int start = i * chunkSize;
            int count = min(chunkSize, n - start);
            int blocks = (count + threadsPerBlock - 1) / threadsPerBlock;
            
            sumChunkKernel<<<blocks, threadsPerBlock, threadsPerBlock * sizeof(float)>>>(
                data, start, count, partialSums + i);
        }
        
        cudaDeviceSynchronize();
        
        // Combine partial sums
        float total = 0.0f;
        for (int i = 0; i < numChunks; i++) {
            float ps;
            // Read partial sum (this is simplified - in practice use another kernel)
            total += partialSums[i];
        }
        
        *result = total;
        cudaFree(partialSums);
        
        printf("CDP Sum Result: %f\n", total);
    }
}

void exercise1_parallel_sum() {
    printf("\n=== Exercise 1: Parallel Sum with CDP ===\n");
    
    const int N = 1024;
    const int NUM_CHUNKS = 4;
    
    float *h_data = new float[N];
    float h_result = 0.0f;
    
    // Initialize data
    float expected = 0.0f;
    for (int i = 0; i < N; i++) {
        h_data[i] = 1.0f;  // Simple case: all ones
        expected += h_data[i];
    }
    
    float *d_data, *d_result;
    CHECK_CUDA(cudaMalloc(&d_data, N * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_result, sizeof(float)));
    CHECK_CUDA(cudaMemcpy(d_data, h_data, N * sizeof(float), cudaMemcpyHostToDevice));
    
    parallelSumKernel<<<1, 1>>>(d_data, N, NUM_CHUNKS, d_result);
    CHECK_CUDA(cudaDeviceSynchronize());
    
    CHECK_CUDA(cudaMemcpy(&h_result, d_result, sizeof(float), cudaMemcpyDeviceToHost));
    
    printf("Expected: %f, Got: %f\n", expected, h_result);
    
    delete[] h_data;
    cudaFree(d_data);
    cudaFree(d_result);
}

// =============================================================================
// Exercise 2: Hierarchical Reduction with CDP
// =============================================================================

__global__ void hierarchicalReduceKernel(float* data, int n, int depth) {
    // TODO: Implement hierarchical reduction
    // Base case: n <= threshold, do simple sum
    // Recursive case: reduce by factor of 4, launch child
    
    if (n <= 4 || depth >= 4) {
        // Base case: simple sum
        if (threadIdx.x == 0) {
            float sum = 0.0f;
            for (int i = 0; i < n; i++) {
                sum += data[i];
            }
            data[0] = sum;
            printf("Depth %d: Base case reduction of %d elements = %f\n", depth, n, sum);
        }
        return;
    }
    
    // Reduce by factor of 4
    int newN = (n + 3) / 4;
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    
    if (tid < newN) {
        float sum = 0.0f;
        for (int i = 0; i < 4 && tid * 4 + i < n; i++) {
            sum += data[tid * 4 + i];
        }
        data[tid] = sum;
    }
    __syncthreads();
    
    if (tid == 0) {
        hierarchicalReduceKernel<<<1, min(newN, 256)>>>(data, newN, depth + 1);
        cudaDeviceSynchronize();
    }
}

void exercise2_hierarchical_reduction() {
    printf("\n=== Exercise 2: Hierarchical Reduction ===\n");
    
    const int N = 256;
    float *h_data = new float[N];
    
    float expected = 0.0f;
    for (int i = 0; i < N; i++) {
        h_data[i] = 1.0f;
        expected += h_data[i];
    }
    
    float *d_data;
    CHECK_CUDA(cudaMalloc(&d_data, N * sizeof(float)));
    CHECK_CUDA(cudaMemcpy(d_data, h_data, N * sizeof(float), cudaMemcpyHostToDevice));
    
    hierarchicalReduceKernel<<<1, 256>>>(d_data, N, 0);
    CHECK_CUDA(cudaDeviceSynchronize());
    
    float result;
    CHECK_CUDA(cudaMemcpy(&result, d_data, sizeof(float), cudaMemcpyDeviceToHost));
    
    printf("Expected: %f, Got: %f\n", expected, result);
    
    delete[] h_data;
    cudaFree(d_data);
}

// =============================================================================
// Exercise 3: Dynamic Work Distribution
// =============================================================================

__global__ void processWorkItem(int* data, int workId, int size) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    if (tid < size) {
        data[tid] = data[tid] * 2 + workId;
    }
}

__global__ void dynamicWorkDistribution(int* data, int* workSizes, int numItems) {
    // TODO: Discover work at runtime and launch children
    // Only process items with size > threshold
    
    if (threadIdx.x == 0 && blockIdx.x == 0) {
        int offset = 0;
        int processed = 0;
        
        for (int i = 0; i < numItems; i++) {
            int size = workSizes[i];
            
            // Runtime decision: only process if size > 10
            if (size > 10) {
                int blocks = (size + 127) / 128;
                processWorkItem<<<blocks, 128>>>(data + offset, i, size);
                processed++;
                printf("Processing work item %d with size %d\n", i, size);
            } else {
                printf("Skipping work item %d (size %d too small)\n", i, size);
            }
            
            offset += size;
        }
        
        cudaDeviceSynchronize();
        printf("Processed %d of %d work items\n", processed, numItems);
    }
}

void exercise3_dynamic_work() {
    printf("\n=== Exercise 3: Dynamic Work Distribution ===\n");
    
    int h_workSizes[] = {5, 50, 8, 100, 3, 200};
    int numItems = 6;
    int totalSize = 0;
    for (int i = 0; i < numItems; i++) totalSize += h_workSizes[i];
    
    int *h_data = new int[totalSize];
    for (int i = 0; i < totalSize; i++) h_data[i] = 1;
    
    int *d_data, *d_workSizes;
    CHECK_CUDA(cudaMalloc(&d_data, totalSize * sizeof(int)));
    CHECK_CUDA(cudaMalloc(&d_workSizes, numItems * sizeof(int)));
    CHECK_CUDA(cudaMemcpy(d_data, h_data, totalSize * sizeof(int), cudaMemcpyHostToDevice));
    CHECK_CUDA(cudaMemcpy(d_workSizes, h_workSizes, numItems * sizeof(int), cudaMemcpyHostToDevice));
    
    dynamicWorkDistribution<<<1, 1>>>(d_data, d_workSizes, numItems);
    CHECK_CUDA(cudaDeviceSynchronize());
    
    CHECK_CUDA(cudaMemcpy(h_data, d_data, totalSize * sizeof(int), cudaMemcpyDeviceToHost));
    
    printf("Sample results: data[0]=%d, data[10]=%d, data[100]=%d\n",
           h_data[0], h_data[10], h_data[100]);
    
    delete[] h_data;
    cudaFree(d_data);
    cudaFree(d_workSizes);
}

// =============================================================================
// Main
// =============================================================================

int main() {
    printf("CDP Fundamentals Exercises\n");
    printf("==========================\n");
    
    exercise1_parallel_sum();
    exercise2_hierarchical_reduction();
    exercise3_dynamic_work();
    
    printf("\n✅ All CDP exercises completed!\n");
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -rdc=true -lcudadevrt -o cdp_fundamentals_exercises cdp_fundamentals_exercises.cu && ./cdp_fundamentals_exercises

### 🔶 Python/Numba Exercises (Alternative)

## 🎯 Key Takeaways

### The Manager Delegation Pattern Mastered!

| Concept | Manager Analogy | CUDA Implementation |
|---------|-----------------|---------------------|
| **Parent Kernel** | Site manager with authority | Kernel that calls `<<<>>>` |
| **Child Kernels** | Specialized sub-teams | Kernels launched from device |
| **Memory Sharing** | Shared blueprints (global) | Only global/unified memory visible |
| **Sync Point** | "Wait for all sub-teams" | `cudaDeviceSynchronize()` |
| **Device Streams** | Parallel work crews | `cudaStreamCreateWithFlags()` |

### CDP Decision Checklist

```
✅ Use CDP when:
   □ Work is discovered at runtime (data-dependent)
   □ CPU roundtrip latency would hurt performance
   □ Recursive algorithms (divide-and-conquer)
   □ Irregular/adaptive workloads

❌ Avoid CDP when:
   □ Fixed, known workload (just launch from CPU)
   □ Shallow recursion (overhead not worth it)
   □ Memory-bound kernels (launch overhead adds up)
```

### Essential Commands

```bash
# Compilation (REQUIRED for CDP)
nvcc -rdc=true -lcudadevrt my_cdp_code.cu -o my_cdp_code
```

---

## 🚀 Next: Day 2 - Recursive Algorithms with CDP

Tomorrow we'll apply the manager delegation pattern to recursive algorithms like quicksort and tree traversals!